# Scenario Object Tutorial
---

This is a quick tutorial to guide the user through the functionalities of the *Scenario* object. The object has several internal state. So far the **create**, **execute**, **analyze** and **delete** states have been implemented.  Only one argument is required to instantiate a *Scenario* object: the `descriptor` (type `str`) of the scenario, which can either be the scenario id or name. Depending on the descriptor passed, the *Scenario* object will take different state:
* empty string --> **create** state;
* valid scenario id or name --> **analyze** state is activated if the simulation has been successfully ran and the output data have been extracted. Otherwise the state will be set to **execute**.

The architecture of the *Scenario* object is shown below.
![Scenario Workflow](scenario.jpg)

## Analyze State
Let's see what happens when the scenario name or id passed as argument is not found in the scenario list file on the server.

In [ ]:
from powersimdata.scenario.scenario import Scenario

In [ ]:
scenario = Scenario('dummy')

It prints the list of available scenarios. Note that the scenario **id** is unique while the scenario **name** is only unique for a given plan - interconnect combination. If there are multiple scenario sharing the same name in the scenario list file then the above summary will be returned and the user will be asked to provide the identification number in order to select a scenario.

Let's now pick a scenario that exists.

In [ ]:
scenario = Scenario('3')

This *Scenario* object is in the **analyze** state. Note that it is possible to switch state. The **delete** state is accessible.

In [ ]:
print("State name: %s" % scenario.state.name)
print("Allowed state: %s" % scenario.state.allowed)

The description of the scenario can easily be accessed.

In [ ]:
scenario.print_scenario_info()

When a *Scenario* object is created, the change table and grid are automatically loaded. Changes found in the change table are applied to the grid. This means that the capacity of some generators and/or transmission lines will be scaled. The grid used for this scenario can be accessed as follows:

In [ ]:
grid = scenario.state.get_grid()
grid.plant.head(n=10)

And the change table:

In [ ]:
ct = scenario.state.get_ct()
print(ct)

One can sees that the capacity of solar and wind generators in load zones 203, 204, 205, 206 and 207 have been increased by 1.52 and 1.48, respectively. Also, the capacity of some transmission lines has bee increased by a factor 2. In this case the identification number of the branch has been used. The mapping load zone id to load zone name is accessible via the *Grid* object.

In [ ]:
grid.zone

Those are the 16 load zones of the Western Interconnect. One can sees that only the solar and wind generators in California have been scaled. Let's take a look at the transmission lines in the change table:

In [ ]:
grid.branch.loc[ct['branch']['branch_id'].keys()]

Where do these lines start and end?

In [ ]:
for line in ct['branch']['branch_id'].keys():
    print("#%d: %s --> %s" % (line, grid.branch.loc[line].from_zone_name, grid.branch.loc[line].to_zone_name))

Input profiles and output data can be loaded as follows. If the file is not found locally, then it will be downloaded from the server.

In [ ]:
wind = scenario.state.get_wind()
wind.head(n=10)

Let's take a look at infeasibilities. These are period where demand were decreased by some amout in order to make the optimization problem solvable.

In [ ]:
scenario.state.print_infeasibilities()

Below, we plot the original demand profile along with the reduced one for Washington State.

In [ ]:
import matplotlib.pyplot as plt

zone = 201
demand_original = scenario.state.get_demand()
demand_modified = scenario.state.get_demand(original=False)

fig = plt.figure(figsize=(15, 8))
plt.title("%s Demand Profile" % grid.zone[201], fontsize=25)
ax = fig.gca()
ax.grid(color='black', axis='y')
ax.tick_params(which='both', labelsize=20)
ax = demand_original[zone].rename('original').resample('D').sum().plot(ax=ax, color='red', legend=True, lw=3)
ax = demand_modified[zone].rename('modified').resample('D').sum().plot(ax=ax, color='blue', legend=True, lw=3)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], frameon=2, prop={'size': 18})
ax.set_xlabel('')
ax.set_ylabel('Load (MWh)', fontsize=22)
plt.show()

print("Original demand: %d (MWh)" % demand_original[201].sum())
print("Reduced demand: %d (MWh)" % demand_modified[zone].sum())

## Create State
Let's create a scenario now!

In [ ]:
new_scenario = Scenario('')
print("State name: %s" % new_scenario.state.name)
print("Allowed state: %s" % new_scenario.state.allowed)

In [ ]:
new_scenario.print_scenario_info()

To create a scenario, one needs to set the interconnect, the plan / name, start date / end date / interval, base_demand, base_hydro, base_solar and base_wind profiles version and, optionally, scale the capacity of generators and/or transmission lines. Let's do so.

In [ ]:
new_scenario.state.set_builder(['Western'])

The Western interconnect is loaded and the already existing plans for the Western interconnect are given. Also, a list of the available profiles for the chosen interconnect is printed. Let's name our scenario.

In [ ]:
new_scenario.state.builder.set_name('ca2045', 'ca2020')

Oops, we can't do that. Let's pick a different name for our new scenario. Also, let's set the start date, end date and interval. Note that the interval size must satisfy the following condition: end date - start date (in hours) % interval (in hours) = 0

In [ ]:
new_scenario.state.builder.set_name('test', 'dummy')
new_scenario.state.builder.set_time('2016-08-01 00:00:00', '2016-08-31 23:00:00', '124H') 

Is that time to create the scenario?

In [ ]:
new_scenario.state.create_scenario()

Nope. We still need to set the base profiles and eventually a change table.

In [ ]:
new_scenario.state.builder.set_base_profile('demand', 'v3')
new_scenario.state.builder.set_base_profile('hydro', 'v1')
new_scenario.state.builder.set_base_profile('solar', 'v2')
new_scenario.state.builder.set_base_profile('wind', 'v1')

And increase the capacity of wind farms in Wisconsin.

In [ ]:
new_scenario.state.builder.change_table.scale_plant_capacity('wind', {'Winsconsin': 5})

Right, Wisconsin is not part of the Western interconnect.

In [ ]:
# scale capacity of solar plants in WA and AZ by 5 and 2.5, respectively
scenario.state.builder.change_table.scale_plant_capacity('solar',
                                                         zone={'Washington': 5,
                                                               'Arizona': 2.5})
# scale capacity of wind farms in OR and MT by 1.5 and 2, respectively
scenario.state.builder.change_table.scale_plant_capacity('wind',
                                                         zone={'Oregon': 1.5,
                                                               'Montana': 2})
# scale capacity of solar plants in NV and WY by 2
scenario.state.builder.change_table.scale_branch_capacity(zone={'Nevada': 2,
                                                                'Wyoming': 2})
print(scenario.state.builder.change_table.ct)  # print change table

Let's create the scenario now.

In [ ]:
new_scenario.state.create_scenario()

In [ ]:
print("State name: %s" % scenario.state.name)
print("Allowed state: %s" % scenario.state.allowed)